# Preparation 正则表达式

## p.0 

正则表达式是一个特殊的字符序列，它能帮助你方便的检查一个字符串是否与某种模式匹配。

Python 自1.5版本起增加了re 模块，它提供 Perl 风格的正则表达式模式。

In [1]:
import re

正则表达式的大致匹配过程是：
1. 依次拿出表达式和文本中的字符比较，
2. 如果每一个字符都能匹配，则匹配成功；一旦有匹配不成功的字符则匹配失败。
3. 如果表达式中有量词或边界，这个过程会稍微有一些不同。

**r** ：Python 中字符串的前导 r 代表原始字符串标识符，该字符串中的特殊符号不会被转义，适用于正则表达式中繁杂的特殊符号表示。 因此 r"\n" 表示包含 '\' 和 'n' 两个字符的字符串，而 "\n" 则表示只包含一个换行符的字符串。

In [19]:
print("\\n") # 输出 \n
print(r"\n") #输出 \n

\n
\n


## p.1 常量

re模块中有9个常量，常量的值都是int类

常用：I,A,S,M,X
- A = ASCII         &emsp;&emsp;&emsp;&emsp;只匹配ASCII码，无Uinicode
- I = IGNORECASE    &emsp;大小写忽略匹配
- L = LOCALE        &emsp;&emsp;&emsp;(不推荐使用)由系统当前语言区域决定的匹配方式
- U = UNICODE       &emsp;&emsp;匹配unicode编码支持的字符 (而Py3默认字符串已经是Unicode)
- M = MULTILINE     &emsp;多行模式，支持匹配行开头
- S = DOTALL        &ensp;&emsp;&emsp;.匹配所有模式，包括换行符\n一起匹配，而不分开
- X = VERBOSE       &emsp;&emsp;详细模式
- T = TEMPLATE      
- ~ ~ DEBUG         &emsp;&emsp;&emsp;显示编译时的debug信息

`verbose`: (adj.) 冗长的，罗嗦的

先不管语法，看看例子：

In [18]:
text = 'Renna blue'
pattern = 'RENNA'
print('默认模式: ', re.findall(pattern, text))
print('Ignorecase: ', re.findall(pattern, text, re.I))

text = 'se人形のRenna'
pattern = r'\w+'
print('Unicode(default): ', re.findall(pattern, text))
print('ASCII: ', re.findall(pattern, text, re.A))

text = 'Renna \n blue'
pattern = r'.*'
print('默认模式: ', re.findall(pattern, text))
print('Dotall: ', re.findall(pattern, text, re.S))

text = 'Renna\nblue' #注意若为 'Renna \n blue' 行开头是' blue'
pattern = r'^blue'
print('默认模式: ', re.findall(pattern, text))
print('Multiline: ', re.findall(pattern, text, re.M))

text = 'Renna blue'
pattern = r"""^Renna    # Subject
            \ blue     # Adjective
           """
print('默认模式: ', re.findall(pattern, text))
print('Verbose: ', re.findall(pattern, text, re.X))

默认模式:  []
Ignorecase:  ['Renna']
Unicode(default):  ['se人形のRenna']
ASCII:  ['se', 'Renna']
默认模式:  ['Renna ', '', ' blue', '']
Dotall:  ['Renna \n blue', '']
默认模式:  []
Multiline:  ['blue']
默认模式:  []
Verbose:  ['Renna blue']


## p.2 **pattern**

### 单次字符匹配

本部分介绍书写函数中的pattern参数时常用的方法。

所举均以re.match()为例，可以先学习p.3.2 match()函数

| Character | Function |
| :---:     | :---:    |
| .         | 通配符(except \n)|
| [ ]       | 匹配[ ]中字符 |
| \d        | 匹配数字0~9 |
| \D        | 匹配非数字 |
| \s        | 匹配空白(Tab/空格) |
| \S        | 匹配非空白字符 |
| \w        | 匹配单词字符(a-z,A-Z,0-9,_)|
| \W        | 匹配非单词字符 |

In [ ]:
import re
ret = re.match(".","M")
print(ret.group())
ret = re.match("t.o","two")
print(ret.group())

# 如果hello的⾸字符⼩写，那么正则表达式需要⼩写的 h (反之同理)
ret = re.match("h","hello Python")
print(ret.group())
# ⼤⼩写h都可以的情况
ret = re.match("[hH]","Hello Python")
print(ret.group())
ret = re.match("[hH]ello Python","Hello Python")
print(ret.group())

# 匹配0到9的多种写法
ret = re.match("[0123456789]Hello Python","7Hello Python")
print(ret.group())
ret = re.match("[0-9]Hello Python","7Hello Python")
print(ret.group())

# 匹配0到3和5-9
ret = re.match("[0-35-9]Hello Python","7Hello Python")
print(ret.group())
ret = re.match("[0-35-9]Hello Python","4Hello Python")
print(ret)

ret = re.match("嫦娥\d号","嫦娥1号发射成功")
print(ret.group())
ret = re.match("嫦娥2\D","嫦娥2号发射成功")
print(ret.group())

M
two
h
H
Hello Python
7Hello Python
7Hello Python
7Hello Python
None
嫦娥1号
嫦娥2号


### 多次字符匹配

| Character | Function | Exp Instance | Complete matching string |
| :---:     | :---:    | :---:        | :---:                    |
| *         | 前一字符$0~\infin$次 | abc* | abc,abcc,abccc... |
| +         | 前一字符$1~\infin$次 | abc+ | abcc,abccc... |
| ?         | 前一字符1或0次 | abc? | ab,abc |
| {m}       | 前一字符m次 | ab{2}c | abbc |
| {m,n}     | 前一字符m至n次 | ab{1,2}c | abc,abbc |

> 匹配优先取最长；<br>
> {,n}同于{0,n}，{m,}同于{m,$\infin$}

In [ ]:
import re
# 匹配出，⼀个字符串第⼀个字⺟为⼤写字符，后⾯都是⼩写字⺟并且这些⼩写字⺟可有可⽆
ret = re.match("[A-Z][a-z]*","M")
print(ret.group())
ret = re.match("[A-Z][a-z]*","MnnM")
print(ret.group())
ret = re.match("[A-Z][a-z]*","Aabcdef")
print(ret.group())

#匹配出，变量名是否有效（以字母开头视为有效）
names = ["name1", "_name", "2_name", "__name__"]
for name in names:
    ret = re.match("[a-zA-Z_]+[\w]*",name)
    if ret:
        print("变量名 %s 符合要求" % ret.group())
    else:
        print("变量名 %s ⾮法" % name)

#匹配出，0到99之间的数字
ret = re.match("[1-9]?[0-9]","7")
print(ret.group())
ret = re.match("[1-9]?\d","33")
print(ret.group())
# 下例中，原意是想排除0匹配9，实际匹配到了0
# 需要用 $ (下一段介绍)
ret = re.match("[1-9]?\d","09")
print(ret.group())

ret = re.match("[a-zA-Z0-9_]{6}","12a3g45678")
print(ret.group())
#匹配出，8到20位的密码，可以是⼤⼩写英⽂字⺟、数字、下划线
ret = re.match("[a-zA-Z0-9_]{8,20}","1ad12f23s_HAB34455ff66")
print(ret.group())
ret = re.match("[a-zA-Z0-9_]{,5}","1ad12f23s_HAB34455ff66")
print(ret.group())
ret = re.match("[a-zA-Z0-9_]{5,}","1ad12f23s_HAB34455ff66")
print(ret.group())

M
Mnn
Aabcdef
变量名 name1 符合要求
变量名 _name 符合要求
变量名 2_name ⾮法
变量名 __name__ 符合要求
7
33
0
12a3g4
1ad12f23s_HAB34455ff
1ad12
1ad12f23s_HAB34455ff66


### 首尾匹配

| Character | Function |
| :---:     | :---:    |
| ^         | 匹配字符串开头 |
| $         | 匹配字符串结尾 |

In [41]:
import re
email_list = ["xiaoWang@163.com", "xiaoWang@163.comheihei", ".com.xiaowang@qq.com"]
for email in email_list:
    ret = re.match("[\w]{4,20}@163\.com$", email)
    if ret:
        print("%s 是符合规定的邮件地址,匹配后的结果是:%s" % (email, ret.group()))
    else:
        print("%s 不符合要求" % email)

xiaoWang@163.com 是符合规定的邮件地址,匹配后的结果是:xiaoWang@163.com
xiaoWang@163.comheihei 不符合要求
.com.xiaowang@qq.com 不符合要求


### pattern分组

| Character  | Function |
| :---:      | :---:    |
| \|         | 或，匹配左右任意一个表达式 |
| ( )        | ( )中字符作为一个组 |
| \num       | 引⽤分组num匹配到的字符串 |
| (?P\<name\>) | 分组起别名，匹配到的子串组在外部是通过定义的 name 来获取的 |
| (?P=name)  | 引⽤别名为name分组匹配到的字符串 |

In [43]:
# | 举例
# 匹配出0-100之间的数字
# 用 $ 从后往前匹配，避免误判以 0 开头的非法数字
import re
ret = re.match("[1-9]?\d$|100","8")
print(ret.group()) # 8
ret = re.match("[1-9]?\d$|100","78")
print(ret.group()) # 78
ret = re.match("[1-9]?\d$|100","08")
print(ret) # 不是0-100之间
ret = re.match("[1-9]?\d$|100","100")
print(ret.group()) # 100

8
78
None
100


In [44]:
# () 分组举例
#需求：匹配出163、126、qq邮箱
ret = re.match("\w{4,20}@163\.com", "test@163.com")
print(ret.group()) # test@163.com
ret = re.match("\w{4,20}@(163|126|qq)\.com", "test@126.com")
print(ret.group()) # test@126.com
ret = re.match("\w{4,20}@(163|126|qq)\.com", "test@qq.com")
print(ret.group()) # test@qq.com
ret = re.match("\w{4,20}@(163|126|qq)\.com", "test@gmail.com")
if ret:
    print(ret.group())
else:
    print("不是163、126、qq邮箱") # 不是163、126、qq邮箱
#不是以4、7结尾的⼿机号码(11位)
tels = ["13100001234", "18912344321", "10086", "18800007777"]
for tel in tels:
    ret = re.match("1\d{9}[0-35-68-9]", tel)
    if ret:
        print(ret.group())
    else:
        print("%s 不是想要的⼿机号" % tel)
#提取区号和电话号码
ret = re.match("([^-]*)-(\d+)","010-12345678")
print(ret.group())
print(ret.group(1))
print(ret.group(2))

test@163.com
test@126.com
test@qq.com
不是163、126、qq邮箱
13100001234 不是想要的⼿机号
18912344321
10086 不是想要的⼿机号
18800007777 不是想要的⼿机号
010-12345678
010
12345678


**\number** 用法：

&emsp;&emsp;匹配数字代表的组合。每个括号是一个组合，组合从1开始编号。
<br> 注意：**组合编号是自动的**，出现()后，期内组合自动获得编号。

&emsp;&emsp;比如 (.+) \1 匹配 'the the' 或者 '55 55', 但不会匹配 'thethe' (注意组合后面的空格)。这个特殊序列只能用于匹配前面99个组合。
<br>&emsp;&emsp;如果 number 的第一个数位是0， 或者 number 是三个八进制数，它将不会被看作是一个组合，而是八进制的数字值。
<br>&emsp;&emsp;在 '[' 和 ']' 字符集合内，任何数字转义都被看作是字符。

In [46]:
import re
# 正确的理解思路：如果在第⼀对<>中是什么，按理说在后⾯的那对<>中就应该是什么。
# 通过引⽤分组中匹配到的数据即可，但是要注意是元字符串，即类似 r""这种格式。
ret = re.match(r"<([a-zA-Z]*)>\w*</\1>", "<html>hh</html>")
# 因为2对<>中的数据不⼀致，所以没有匹配出来
test_label = ["<html>hh</html>","<html>hh</htmlbalabala>"]
for label in test_label:
    ret = re.match(r"<([a-zA-Z]*)>\w*</\1>", label)
    if ret:
        print("%s 这是一对正确的标签" % ret.group())
    else:
        print("%s 这是⼀对不正确的标签" % label)


labels = ["<html><h1>www.itcast.cn</h1></html>", "<html><h1>www.itcast.cn</h2></html>"]
for label in labels:
    ret = re.match(r"<(\w*)><(\w*)>.*</\2></\1>", label)
    if ret:
        print("%s 是符合要求的标签" % ret.group())
    else:
        print("%s 不符合要求" % label)

<html>hh</html> 这是一对正确的标签
<html>hh</htmlbalabala> 这是⼀对不正确的标签
<html><h1>www.itcast.cn</h1></html> 是符合要求的标签
<html><h1>www.itcast.cn</h2></html> 不符合要求


**(?P\<name\>) (?P=name)**

一个用于标记，一个用于在同一个正则表达式中复用

相当于自定义名取代自动的\number编号，name对应的内容可随意替换

In [47]:
import re
ret = re.match(r"<(?P<name1>\w*)><(?P<name2>\w*)>.*</(?P=name2)></(?P=name1)>","<html><h1>www.itcast.cn</h1></html>")
print(ret.group())
ret = re.match(r"<(?P<name1>\w*)><(?P<name2>\w*)>.*</(?P=name2)></(?P=name1)>","<html><h1>www.itcast.cn</h2></html>")
print(ret)

<html><h1>www.itcast.cn</h1></html>
None


## p.3 函数

### p.3.1 函数一览

查找并返回一个匹配项的函数有3个

- **search**： &emsp;查找任意位置的匹配项
- **match**：  &emsp;必须从字符串开头匹配
- **fullmatch**：    整个字符串与正则完全匹配


查找并返回多个匹配项的函数有2个

- **findall**： &ensp;从字符串任意位置查找，返回一个**列表**
- **finditer**：  从字符串任意位置查找，返回一个**迭代器**

编译正则对象

- **compile**
- **template**

分割与替换：

- **split** : &ensp;匹配给定正则式对应的字符以分割字符串
- **sub** :   &emsp;匹配字符并替换
- **subn** :  &ensp;匹配字符并替换(元组)


### p.3.2 **.match(), .search(), .fullmatch()**

(补充)**.group()** 方法：

&emsp;group()用来提出分组截获的字符串。group() 同group（0）就是匹配正则表达式整体结果，group(1) 列出第一个括号匹配部分...类推

**re.match( pattern, string, flags=0 )** :

只能从头开始匹配，失败则返回none.

> pattern: 匹配的正则表达式<br>
> string: 匹配对象字符串<br>
> flags: 标志位，用于控制正则表达式的匹配方式 (如re.A,re.X等)

In [26]:
result = re.match("itcast","itcast.cn")
print(result.group())
result = re.match("itcast","cn.itcast")
print(result)

itcast
None


**re.search( )** : 

扫描整个字符串并返回第一个成功的匹配，如果没有匹配，就返回一个 None。

In [50]:
ret = re.search(r"\d+$", "阅读114514次数为1919")
print(ret.group())

114514
1919


**re.fullmatch( )** : 

只有整个字符串匹配时才成功（严格match）。

In [54]:
ret = re.fullmatch(r"\d+", "114514abc1919")
print(ret)
ret = re.fullmatch(r"\d+\w*", "114514abc1919")
print(ret.group())

None
114514abc1919


### p.3.3 **.findall( ), .finditer( )**

**.findall( ), .finditer( )**

&emsp;&emsp;二者均是在字符串中找到正则表达式所匹配的所有子串，并返回；如果没有找到匹配的，则返回空。区别是前者放回列表，后者返回迭代器。

In [68]:
ret = re.findall(r"\d+", "python = 9999, c = 7890, c++ = 12345")
print(ret)
it = re.finditer(r"\d+", "12a32bc43jf3")
print(type(it))
for match in it:
    print(match.group())

['9999', '7890', '12345']
<class 'callable_iterator'>
12
32
43
3


### p.3.4 **.compile( ), .template( )**

compile 函数用于编译正则表达式，生成一个正则表达式（ Pattern ）对象

template类似，区别？？

In [73]:
pattern = r"\w+"
string = "Reimu"

# 下面两者等价
prog = re.compile(pattern)
result = prog.match(string)

result = re.match(pattern, string)

### p.3.5 **.split( ), .sub( ), .subn( )**

**re.split( pattern, string, maxsplit=0, flags=0 )**: 

用 pattern 分开 string ， maxsplit 表示最多进行分割次数， flags 表示模式

In [ ]:
ret = re.split(r":| ","info:xiaoZhang 33 shandong")
print(ret)

['info', 'xiaoZhang', '33', 'shandong']


**re.sub( pattern, repl, string, count=0, flags=0 )**

repl 是用于替换的字符串或函数，count 为最大替换次数( 非负，0或缺省则替换所有匹配字符串 )

**re.subn( pattern, repl, string[ , count] )**

行为与sub()相同，但是返回一个元组 (字符串, 替换次数)。

In [ ]:
# example: 两种方法使字符串内数字加一
ret = re.sub(r"\d+", '998', "python = 997")
print(ret)

# repl 为函数的情况(不写括号)
# 其含义是将匹配串作为参数传入 该函数，然后得到的函数返回结果作为 repl
def add(temp):
    #int() 参数必须是字符串，类似字节的对象或数字，而不是“re.Match”
    strNum = temp.group()
    num = int(strNum) + 1
    return str(num)
ret = re.sub(r"\d+", add, "python = 997")
print(ret)
ret = re.sub(r"\d+", add, "python = 99")
print(ret)

python = 998
python = 998
python = 100


In [74]:
pattern = re.compile(r'(\w+) (\w+)')
s = 'i say, hello wORLd!'
print(re.subn(pattern, r'\2 \1', s)) # 将词序调换
def func(m):
    return m.group(1).title() + ' ' + m.group(2).title()
print(re.subn(pattern, func, s))

 # title(): 将字符串标题化（即大小首字母，其余小写）

('say i, wORLd hello!', 2)
('I Say, Hello World!', 2)


### p.3.6 其余（此处不写）

## p.4 贪婪与否

Python⾥数量词默认是贪婪的（在少数语⾔⾥也可能是默认⾮贪婪），总是尝试匹配尽可能多的字符；⾮贪婪则相反，总是尝试匹配尽可能少的字符。

例如：正则表达式”ab*”如果用于查找”abbbc”，将找到”abbb”。而如果使用非贪婪的数量词”ab*?”，将找到”a”。

注：我们一般使用非贪婪模式来提取。

在"*","?","+","{m,n}"后⾯加上？，使贪婪变成⾮贪婪。

In [79]:
import re
s="This is a number 114-514-1919-810"
#正则表达式模式中使⽤到通配字，那它在从左到右的顺序求值时，会尽量“抓取”满⾜匹配最⻓字符串，在我们上⾯的例⼦⾥⾯，“.+”会从字符串的启始处抓取满⾜模式的最⻓字符，其中包括我们想得到的第⼀个整型字段的中的⼤部分，“\d+”只需⼀位字符就可以匹配，所以它匹配了数字“4”，⽽“.+”则匹配了从字符串起始到这个第⼀位数字4之前的所有字符
r=re.match(".+(\d+-\d+-\d+-\d+)",s)
print(r.group(1))
#⾮贪婪操作符“？”，这个操作符可以⽤在"*","+","?"的后⾯，要求正则匹配的越少越好
r=re.match(".+?(\d+-\d+-\d+-\d+)",s)
print(r.group(1))

print(re.match(r"aa(\d+)","aa2343ddd").group(1))
print(re.match(r"aa(\d+?)","aa2343ddd").group(1))
print(re.match(r"aa(\d+)ddd","aa2343ddd").group(1))
print(re.match(r"aa(\d+?)ddd","aa2343ddd").group(1))

test_str="<img data-original=https://rpic.douyucdn.cn/appCovers/2016/11/13/1213973.jpg>"
ret = re.search(r"https://.*?.jpg", test_str)
print(ret.group())

4-514-1919-810
114-514-1919-810
2343
2
2343
2343
https://rpic.douyucdn.cn/appCovers/2016/11/13/1213973.jpg


# **1.3 re: 正则表达式** (unfinished)


## 已掌握内容1

Preparation 介绍得过于详细了。。。

start( ) 和 end( ) 方法用于提供索引

In [1]:
# CODE LIST 1-16
import re

pattern = 'this'
text = 'Does this text match the pattern?'

match = re.search(pattern, text)

s = match.start()
e = match.end()

print('Found "{}"\nin "{}"\nfrom {} to {} ("{}")'.format(
    match.re.pattern, match.string, s, e, text[s:e]))


Found "this"
in "Does this text match the pattern?"
from 5 to 9 ("this")


直接使用已编译表达式可以避免与缓存查找相关的开销

In [2]:
# CODE LIST 1-17
import re

# Precompile the patterns
regexes = [
    re.compile(p)
    for p in ['this', 'that']
]
text = 'Does this text match the pattern?'

print('Text: {!r}\n'.format(text))

for regex in regexes:
    print('Seeking "{}" ->'.format(regex.pattern),
          end=' ')

    if regex.search(text):
        print('match!')
    else:
        print('no match')


Text: 'Does this text match the pattern?'

Seeking "this" -> match!
Seeking "that" -> no match


> `print(*objects, sep=' ', end='\n', file=sys.stdout, flush=False)`
> 
> para sep : 指定分隔符<br>
> para end : 指定结尾符
>
> `format( )`
>
> 类似于 `%s ... %` 的格式控制方法

多重匹配

In [3]:
# CODE LIST 1-18, 1-19
import re

text = 'abbaaabbbbaaaaa'

pattern = 'ab'

for match in re.findall(pattern, text):
    print('Found {!r}'.format(match))

for match in re.finditer(pattern, text):
    s = match.start()
    e = match.end()
    print('Found {!r} at {:d}:{:d}'.format(
        text[s:e], s, e))

Found 'ab'
Found 'ab'
Found 'ab' at 0:2
Found 'ab' at 5:7


## 模式语法